In [ ]:


pres=unit['pressure_dbar'].values.copy()
unit['profile_index'] = xr.DataArray(np.full(pres.shape[0],np.nan), dims=['time'])
# remove surface values (above .5 m)
pres[ np.where(pres<0.5)[0] ] = np.nan
# localise ~nan pressure
pres_idx=np.where(~np.isnan(pres))[0]

# 10-20 shallow points have two identical pressure measurements, potentially separated by nans. 
# Remove latest measurement, otherwise, inflection points are missed.
idx_rm=np.where(np.diff(pres[pres_idx])==0)[0]
pres[pres_idx[idx_rm]]=np.nan
# relocalise ~nan pressure without identical pressure measurements.
pres_idx=np.where(~np.isnan(pres))[0]

# Initial index based on 1st profile (398:dive or 409:climb).
init_index=[1 if np.diff(pres[pres_idx[1:3]])>0 else 1.5][0]
for i in range(pres_idx[1],pres.shape[0]-1):
    # Localise inflection point for 3 valid depths: i.e. sign((p_{z+1}-p_{z})*(p_{z}-p_{z-1}))<0
    if ~np.isnan(pres[i]) :
        idxp=np.where(pres_idx==i)[0][0]
        p0=pres_idx[idxp-1]
        p1=pres_idx[idxp+1]
        if np.sign((pres[p1]-pres[i])*(pres[i]-pres[p0])) == -1:
            init_index+=.5
        unit['profile_index'][i]=init_index

idx_c = np.where( unit['profile_index'] % 1 == .5)[0] # climb indices
idx_d = np.where( unit['profile_index'] % 1 == 0)[0] # dive indices
